# Learn Pydantic: Interactive Guide

Welcome! This notebook will guide you through **Pydantic**, the most widely used data validation library for Python.

## What we will cover:
1.  **Basics**: Models and automatic type conversion
2.  **Fields**: Default values and constraints
3.  **Validation**: Handling errors
4.  **Settings**: Managing environment variables
5.  **Advanced**: Nested models and custom validators

## 1. The Basics

A Pydantic model is a class that inherits from `BaseModel`. You define fields with type hints.

In [1]:
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int
    is_active: bool = True  # Default value

# Create an instance
user = User(name="Alice", age=30)
print(f"User created: {user}")

# Access fields
print(f"Name: {user.name}, Age: {user.age}")

User created: name='Alice' age=30 is_active=True
Name: Alice, Age: 30


### Automatic Type Conversion
Pydantic tries to convert inputs to the declared types (e.g., string "30" becomes integer 30).

In [ ]:
# Passing 'age' as a string
user_converted = User(name="Bob", age="25") 

print(f"Original input age: '25' (str)")
print(f"Stored age: {user_converted.age} ({type(user_converted.age)})\n")
print(user_converted.model_dump()) # Viewing as a dictionary

## 2. Validation Errors
If conversion fails, Pydantic raises a `ValidationError`.

In [2]:
from pydantic import ValidationError

try:
    # 'invalid' cannot be converted to an integer
    User(name="Charlie", age="invalid")
except ValidationError as e:
    print("Validation Error Caught:")
    print(e)

Validation Error Caught:
1 validation error for User
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='invalid', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


## 3. Field Customization
Use the `Field` function to add metadata and validation constraints like `min_length`, `max_value`, etc.

In [ ]:
from pydantic import Field

class Product(BaseModel):
    name: str = Field(min_length=3, description="Name of the product")
    price: float = Field(gt=0, description="Price must be greater than 0")
    tags: list[str] = Field(default=[], description="List of tags")

# Valid product
p1 = Product(name="Laptop", price=999.99)
print(f"Valid Product: {p1}")

# Invalid product (short name, negative price)
try:
    Product(name="TV", price=-50)
except ValidationError as e:
    print("\nInvalid Product Error:")
    print(e)

## 4. Settings Management
Pydantic Settings (`BaseSettings`) is perfect for handling configuration, automatically reading from environment variables.

In [ ]:
import os
from pydantic_settings import BaseSettings

# Simulate environment variables
os.environ["APP_NAME"] = "My Super App"
os.environ["DEBUG_MODE"] = "true"
os.environ["API_KEY"] = "secret-123"

class AppSettings(BaseSettings):
    app_name: str
    debug_mode: bool = False
    api_key: str
    max_retries: int = 3  # Default if not in env

settings = AppSettings()
print("Loaded Settings from Environment:")
print(settings.model_dump())

## 5. Advanced Patterns

### Nested Models
You can use models as types inside other models.

In [ ]:
class Address(BaseModel):
    city: str
    zip_code: str

class UserProfile(BaseModel):
    username: str
    address: Address  # Nested model

data = {
    "username": "dave_01",
    "address": {
        "city": "New York",
        "zip_code": "10001"
    }
}

profile = UserProfile(**data)
print(f"Nested City: {profile.address.city}")

### Custom Validators
Use `@field_validator` for logic that standard constraints can't handle.

In [ ]:
from pydantic import field_validator

class Signup(BaseModel):
    username: str
    password: str
    
    @field_validator('username')
    @classmethod
    def must_be_alphanumeric(cls, v: str) -> str:
        if not v.isalnum():
            raise ValueError('Username must be alphanumeric')
        return v

try:
    Signup(username="user_name!", password="pass123")
except ValidationError as e:
    print("Custom Validation Error:")
    print(e)

## Summary
You've covered:
- Creating Models
- Automatic Type Conversion
- Validation & Constraints
- Settings Management
- Nested Models & Custom Logic

**Next Steps:** Try modifying these examples or creating a model for your own data!